In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import numpy as np
import sys
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

2025-03-03 10:34:58.755116: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 10:34:58.760828: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-03 10:34:58.778358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741019698.806861 2781353 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741019698.815633 2781353 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [18]:
input_shape = (100, 100, 1)
num_classes = 12
num_heads = 4
embed_dim = 32
ff_dim = 64

In [3]:
#Cargar datos de entrenamiento
training_set = sys.argv[0] = "/home/arthur1/Documentos/GitHub/Prueba/Datos/Salida/Datos_entrenamiento_TCGA_matriz.npy" ### "Input data npy file" 
x_sample = np.load ( training_set )
x_train = x_sample.reshape ((x_sample.shape[0], 100, 100,1)) # No. de muestras, Longitud, Ancho, No. de filtros
# Cargar etiquetas de los datos de entrenamiento
training_label = sys.argv[0] = "/home/arthur1/Documentos/GitHub/Prueba/Datos/Salida/Datos_entrenamiento_TCGA_etiquetas.npy" ### "Input label npy file" 
y_label = np.load ( training_label )
y_train = to_categorical(y_label, num_classes=12)

In [4]:
# Carga de datos de validacion
testing_set = sys.argv[0] = "/home/arthur1/Documentos/GitHub/Prueba/Datos/Salida/Datos_validacion_TCGA_matriz.npy"
test_sample = np.load ( testing_set )
x_test = test_sample.reshape ( (test_sample.shape[0], 100, 100, 1) ) # no. of samples, x pixels, y pixels, no. of files

# Carga de etiquetas de datos de validacion
testing_label = sys.argv[0] = "/home/arthur1/Documentos/GitHub/Prueba/Datos/Salida/Datos_validacion_TCGA_etiquetas.npy"
test_label = np.load ( testing_label )
test_label_compare = test_label
y_test = to_categorical (test_label,num_classes=12 )

In [19]:
# Extracción de características con CNN
cnn_inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(cnn_inputs)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
cnn_outputs = layers.Flatten()(x)
cnn_model = models.Model(cnn_inputs, cnn_outputs)

In [20]:
# Reformateo para Transformer
transformer_inputs = layers.Input(shape=(cnn_model.output_shape[1],))
x = layers.Reshape((-1, 1))(transformer_inputs)


In [21]:
# Transformer
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
x = layers.Add()([x, attention_output])
x = layers.LayerNormalization(epsilon=1e-6)(x)
ffn_output = layers.Dense(ff_dim, activation='relu')(x)
ffn_output = layers.Dense(cnn_model.output_shape[1])(ffn_output)
x = layers.Add()([x, ffn_output])
x = layers.LayerNormalization(epsilon=1e-6)(x)
transformer_outputs = layers.GlobalAveragePooling1D()(x)
transformer_model = models.Model(transformer_inputs, transformer_outputs)


In [22]:
# Clasificador
inputs = layers.Input(shape=input_shape)
cnn_output = cnn_model(inputs)
transformer_output = transformer_model(cnn_output)
outputs = layers.Dense(num_classes, activation='softmax')(transformer_output)
model = models.Model(inputs, outputs)

In [23]:
# Compilación
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
# Entrenamiento
model.fit(x_train, y_train, epochs=10)

Epoch 1/10


2025-03-03 10:41:40.175470: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 819200000000 exceeds 10% of free system memory.
2025-03-03 10:41:40.175689: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at einsum_op_impl.h:532 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[32,4,40000,40000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu


ResourceExhaustedError: Graph execution error:

Detected at node functional_8_1/functional_7_1/multi_head_attention_2_1/einsum/Einsum defined at (most recent call last):
  File "/home/arthur1/.pyenv/versions/3.9.10/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/arthur1/.pyenv/versions/3.9.10/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/arthur1/Documentos/GitHub/Prueba/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/arthur1/Documentos/GitHub/Prueba/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/arthur1/Documentos/GitHub/Prueba/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/arthur1/Documentos/GitHub/Prueba/.venv/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/arthur1/.pyenv/versions/3.9.10/lib/python3.9/asyncio/base_events.py", line 596, in run_fo